In [1]:
import pandas as pd
from typing import Callable, List, Optional
from babydragon.memory.indexes.memory_index import MemoryIndex
from babydragon.memory.indexes.python_index import PythonIndex
from babydragon.chat.memory_chat import FifoChat
import babydragon
import os

In [2]:
import openai

openai.api_key = "your key here"

In [3]:
babydragon_path = os.path.dirname(os.path.abspath(babydragon.__file__))

In [4]:
babyindex = PythonIndex(
    babydragon_path, name="babyd_index", minify_code=False, load=False
)

Creating a new index
Indexing 298 functions and 46 classes


The value # This is the __init__.py file for the package.
 was embedded

The value 
def __init__(self, system_prompt: str = None, user_prompt: str = None):
    """
        Initialize the Prompter with system and user prompts.

        :param system_prompt: A string representing the system prompt.
        :param user_prompt: A string representing the user prompt.
        """
    if system_prompt is None:
        self.system_prompt = DEFAULT_SYSTEM_PROMPT
        self.user_defined_system_prompt = False
    else:
        self.system_prompt = system_prompt
        self.user_defined_system_prompt = True
    if user_prompt is None:
        self.user_prompt = self.default_user_prompt
        self.user_defined_user_prompt = False
    else:
        self.user_prompt = user_prompt
        self.user_defined_user_prompt = True

    self.prompt_func: Callable[[str], Tuple[List[str], str]] = self.one_shot_prompt
 was embedded

The value 
def default_user_prompt(self, message: str) -> str:
    return DEFAULT_USER_PROMPT.format(question=message)
 was embedded

The value 
def one_shot_prompt(self, message: str) -> Tuple[List[str], str]:
    """
        Compose the prompt for the chat-gpt API.

        :param message: A string representing the user message.
        :return: A tuple containing a list of strings representing the prompt and a string representing the marked question.
        """
    marked_question = mark_question(self.user_prompt(message))
    prompt = [mark_system(self.system_prompt)] + [marked_question]
    return prompt, marked_question
 was embedded

...

The value 

class LLMWriter(BaseTask):
    def __init__(
        self,
        index: MemoryIndex,
        path: List[List[int]],
        chatbot: Chat,
        write_func = None,
        context= None,
        task_name="summary",
        max_workers: int = 1,
    ):
        """
        Initialize a LLMWriteTask instance.

        :param index: List of strings representing the queries.
        :param path: List of lists, each sub-list defines a sequence over which the task is executed.
        :param chatbot: Chatbot instance used for executing queries.
        :param max_workers: Maximum number of worker threads (default is 4).
        """
        BaseTask.__init__(self, index, path, max_workers)
        self.chatbot = chatbot
        self.write_func = write_func if write_func else self.llm_response
        self.new_index_name = self.index.name + f"_{task_name}"
        self.context = context

    @staticmethod
    def llm_response(chatbot: Chat, message: str, context = None, id = None):
        return chatbot.reply(message)

    def _execute_sub_task(self, sub_path: List[int]) -> List[str]:
        """
        Execute a sub-task using a separate copy of the chatbot instance.

        :param sub_path: List of indices representing the sub-task's sequence.
        :return: List of strings representing the responses for each query in the sub-task.
        """
        if self.parallel:
            # copy the chatbot instance and resets the memory before making the queries in case of multi-threading
            chatbot_instance = copy.deepcopy(self.chatbot)
        else:
            chatbot_instance = self.chatbot
        if isinstance(self.chatbot, BaseThread):
            chatbot_instance.reset_memory()

        sub_results = {}
        for i in sub_path:
            current_val = self.index.values[i]
            response = self.write_func(chatbot_instance, current_val, self.context, id = i)
            sub_results[i] = response
        return sub_results

    def write(self):
        self.execute_task()
        content_to_write = []
        for sub_result in self.results:
            for index_id, response in sub_result.items():
                content_to_write.append((index_id, response))
        # sort the content to write by index_id
        content_to_write.sort(key=lambda x: x[0])
        self.new_index = MemoryIndex(name=self.new_index_name)
        self.new_index.init_index(values=[x[1] for x in content_to_write])
        self.new_index.save()
        return self.new_index
 was embedded

The value 

class CodeGenerator:
    def __init__(self):
        self.git_memory = None
        self.commit_index = None
        self.contextual_memory = None

    def generate_meta_code(self, user_input):
        # Transform user input into meta-code representation
        pass

    def load_git_memory(self, git_memory):
        # Retrieve GitMemory context
        pass

    def load_commit_context(self, commit_index):
        # Retrieve commit context
        pass

    def load_contextual_resources(self, contextual_memory):
        # Load contextual resources required for code generation
        pass

    def get_current_draft(self, commit_context):
        # Get the current draft code from commit context
        pass

    def generate_modifications(self, draft_code, meta_code, context_resources):
        # Generate modifications to the draft code based on meta-code and context resources
        pass

    def extract_source_code(self, modifications, context_resources):
        # Extract source code from modifications and context resources
        pass

    def apply_modifications(self, draft_code, modifications):
        # Apply modifications to the draft code
        pass

    def validate_draft(self, updated_draft, source_code):
        # Validate the updated draft code and check for consistency
        pass

    def compare_draft_with_objective(self, updated_draft, meta_code):
        # Compare the updated draft code with user's goal based on meta-code
        pass

    def store_draft_to_commit_index(self, commit_index, updated_draft):
        # Store the updated draft code in the commit index
        pass

    def rollback(self, commit_index):
        # Roll back the draft code to the previous state in the commit index
        pass

    def codeGenerationIteration(
        self, userInput, gitMemory, commitIndex, contextualMemory
    ):
        metaCode = self.generateMetaCode(userInput)

        # Concurrent processing - Step 1
        with concurrent.futures.ThreadPoolExecutor() as executor:
            gitMemoryContext = executor.submit(self.loadGitMemory, gitMemory)
            commitContext = executor.submit(self.loadCommitContext, commitIndex)
            contextResources = executor.submit(
                self.loadContextualResources, contextualMemory
            )

            gitMemoryContext = gitMemoryContext.result()
            commitContext = commitContext.result()
            contextResources = contextResources.result()

        draftCode = self.getCurrentDraft(commitContext)

        # Concurrent processing - Step 2
        with concurrent.futures.ThreadPoolExecutor() as executor:
            modifications = executor.submit(
                self.generateModifications, draftCode, metaCode, contextResources
            )
            sourceCode = executor.submit(
                self.extractSourceCode, modifications, contextResources
            )

            modifications = modifications.result()
            sourceCode = sourceCode.result()

        updatedDraft = self.applyModifications(draftCode, modifications)

        # Concurrent processing - Step 3
        with concurrent.futures.ThreadPoolExecutor() as executor:
            validationResult = executor.submit(
                self.validateDraft, updatedDraft, sourceCode
            )
            comparisonResult = executor.submit(
                self.compareDraftWithObjective, updatedDraft, metaCode
            )

            validationResult = validationResult.result()
            comparisonResult = comparisonResult.result()

        if validationResult and comparisonResult:
            self.storeDraftToCommitIndex(commitIndex, updatedDraft)
        else:
            updatedDraft = self.rollback(commitIndex)

        return updatedDraft
 was embedded

The value 

class PythonMinifier:
    def __init__(self, code: str = None):

        self.code = code
        self.output_code = None

    def minify(self):
        if self.code:
            self.output_code = self.minify_code(self.code)

    def get_minified_code(self):
        if not self.output_code:
            self.minify()
        return self.output_code

    @staticmethod
    def minify_code(code: str) -> str:
        return minify(code)
 was already in the index

The value 

class PythonDocstringExtractor:
    @staticmethod
    def extract_docstring(function_def: cst.FunctionDef) -> str:
        docstring = None

        for stmt in function_def.body.body:
            if isinstance(stmt, cst.SimpleStatementLine):
                for expr in stmt.body:
                    if isinstance(expr, cst.Expr) and isinstance(
                        expr.value, cst.SimpleString
                    ):
                        docstring = expr.value.value.strip('"').strip("'")
                        break
            if docstring is not None:
                break

        if docstring is not None:
            return docstring.strip()
        else:
            function_name = function_def.name.value
            return f"No docstring provided for function '{function_name}'. Please add a docstring to describe this function."
 was already in the index

The value 

class GitMemory(MemoryIndex):
    def __init__(self, username, repo_name):
        super().__init__()
        self.username = username
        self.repo_name = repo_name
        self.parser = GitHubRepoProcessor(username, repo_name)
        self.minifier = PythonMinifier()
        self.docstring_extractor = PythonDocstringExtractor()
        self.directory_parser = None
        self.min_code_index = None
        self.doc_string_index = None
        self.libcst_node_index = None

    def create_code_index(self, base_directory):
        self.directory_parser = self.parser.process_repo(base_directory)
        code_values, code_nodes = self.parser.get_values()
        self.code_index = self.init_index(values=code_values)
        self.code_index.save()

    def create_indexes(self, base_directory):
        self.directory_parser = self.parser.process_repo(base_directory)
        code_values, code_nodes = self.parser.get_values()
        self.code_index = self.init_index(values=code_values)

        min_code_values = []
        doc_string_values = []
        for code_value, code_node in zip(code_values, code_nodes):
            minifier = PythonMinifier(code=code_value)
            min_code = minifier.get_minified_code()
            doc_string = self.docstring_extractor.extract_docstring(code_node)
            min_code_values.append(min_code)
            doc_string_values.append(doc_string)
        self.doc_string_index = self.init_index(values=doc_string_values)
        self.min_code_index = self.init_index(values=min_code_values)
 was embedded

In [5]:
chatbot = FifoChat(
    index_dict={"babyindex": babyindex}, name="babyd_chatbot", max_fifo_memory=2000
)

In [ ]:
babyindex.faiss_query("PythonIndex")

In [10]:
chatbot.reply("Can you explain me memorythread?")

Trying to call OpenAI API...


#### Question: 
 Can you explain me memorythread?

 #### Anwser: 
 Certainly! `memory_thread` appears to be a list object that is used to store memory in various classes. It is initialized and reset in different ways depending on the class in which it is used. For example, in the `FIFOMemoryThread` class, `memory_thread` is an instance attribute that is defined in the `__init__` method and reset with the `reset_memory` method. In the `MemoryIndex` class, `memory_thread` is inherited from a superclass and is accessed using the `__getitem__` method. Additionally, the length of `memory_thread` can be obtained using the `__len__` method defined in various classes that include it. Overall, `memory_thread` seems to be a fundamental aspect of memory storage within the classes utilizing it.

'Certainly! `memory_thread` appears to be a list object that is used to store memory in various classes. It is initialized and reset in different ways depending on the class in which it is used. For example, in the `FIFOMemoryThread` class, `memory_thread` is an instance attribute that is defined in the `__init__` method and reset with the `reset_memory` method. In the `MemoryIndex` class, `memory_thread` is inherited from a superclass and is accessed using the `__getitem__` method. Additionally, the length of `memory_thread` can be obtained using the `__len__` method defined in various classes that include it. Overall, `memory_thread` seems to be a fundamental aspect of memory storage within the classes utilizing it.'

In [ ]:
chatbot.gradio()